# Model training notebook


In [1]:
from LangDataloader import LangDataloader
from LangDataloader import addData
import numpy as np
import tensorflow as tf
import keras_lmu 
import copy
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
defultePlotSize = plt.rcParams['figure.figsize']
oringalFontSize = matplotlib.rcParams['font.size']

2023-06-08 10:26:09.788445: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


We will only load a subset of the total dataset to save time. 

In [2]:
data = LangDataloader("/home/khood/GitHub/STREAM/Prepocessing/processedData/index.csv", "/home/khood/GitHub/STREAM/Prepocessing/processedData")
rawData = []
lastStep = 0
labelToNum = {}
num = 0
for step in list(range(0,len(data),100))[:2]:
    for file in range(len(data))[lastStep:step]:
        print(f"adding {lastStep} to {step} of {len(data)}: {'{0:.0f}%'.format(step/len(data) * 100)}... ",end="\x1b\r")
        label = None
        if data[file][1][0] in labelToNum.keys():
            label = np.array([labelToNum[data[file][1][0]]])
        else:
            labelToNum[data[file][1][0]] = num
            num = num + 1
            label = np.array([labelToNum[data[file][1][0]]])
            
        melspectrogram = np.array([data[file][0]]).flatten()
        melspectrogram = [[i] for i in melspectrogram]
        rawData.append([melspectrogram, label])
        
    lastStep = step
        
# for file in range(len(data))[lastStep:]: # grab the remaning (if the size%stepSize != 0 then there will be some left over)
#     print(f"adding {lastStep} to {len(data)} of {len(data)}: {'{0:.0f}%'.format(step/len(data) * 100)}... ",end="\x1b\r")
#     rawData.append(data[file])
    
rawData[0][0]

[[3.755148272515285e-20],
 [2.568578216875089e-13],
 [8.811583029455505e-06],
 [0.00012983147462364286],
 [0.0001186872468679212],
 [0.00010715652024373412],
 [0.00017705878417473286],
 [0.00019923163927160203],
 [9.357128874398768e-05],
 [1.4269797247834504e-05],
 [1.2840345334552694e-05],
 [1.3032259630563203e-05],
 [2.3437069103238173e-05],
 [2.5691699192975648e-05],
 [5.593668538494967e-05],
 [1.4664220543636475e-05],
 [1.117618739954196e-05],
 [7.151053068810143e-06],
 [1.049544516718015e-05],
 [1.284125119127566e-05],
 [1.91502804227639e-05],
 [1.3268237125885207e-05],
 [5.238910034677247e-06],
 [3.8975489587755874e-05],
 [0.0014837041962891817],
 [0.0024472600780427456],
 [0.0019210706232115626],
 [0.005382259376347065],
 [0.007959770038723946],
 [0.006091925781220198],
 [0.00456344336271286],
 [0.02438884787261486],
 [0.03551292419433594],
 [0.06724800169467926],
 [0.027123361825942993],
 [0.006219853647053242],
 [0.05725622549653053],
 [0.07347908616065979],
 [0.00977116916328

convert targets to numbers classes

In [3]:
len(labelToNum.keys())

80

The last dimension should be time, the second to last should be the n_mels

<p>not flattened shape: (1, 128, 430)
<p>flattened shape: (55040,)

In [4]:
n_pixels = 55040

lmu_layer = keras_lmu.LMU(
    memory_d=1,
    order=256,
    theta=n_pixels,
    hidden_cell=tf.keras.layers.SimpleRNNCell(2750),
    hidden_to_memory=False,
    memory_to_memory=False,
    input_to_hidden=True,
    kernel_initializer="ones",
)

# TensorFlow layer definition
inputs = tf.keras.Input((n_pixels, 1))
lmus = lmu_layer(inputs)
outputs = tf.keras.layers.Dense(len(labelToNum.keys()))(lmus)

# TensorFlow model definition
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="adam",
    metrics=["accuracy"],
)
model.summary()

2023-06-08 10:26:19.297989: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-08 10:26:19.300026: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 55040, 1)]        0         
                                                                 
 lmu (LMU)                   (None, 1)                 5         
                                                                 
 dense (Dense)               (None, 80)                160       
                                                                 
Total params: 165
Trainable params: 165
Non-trainable params: 0
_________________________________________________________________


In [5]:
valid = []
train = []
addData(valid, train, rawData, int(len(rawData)*0.9))
print(f"Train: {len(train)}")
print(f"Valid: {len(valid)}")

Train: 90
Valid: 10


In [6]:
X_train = []
Y_train = []
for i in train:
    X_train.append(np.array(i[0]))
    Y_train.append(i[0][0][0])
X_train=np.array(X_train)
Y_train=np.array(Y_train)

In [7]:
print(f"Y_train size: {len(Y_train)}, X_train size: {len(X_train)}")
Y_train.shape

Y_train size: 90, X_train size: 90


(90,)

In [8]:
Y_train[:10]

array([3.75514827e-20, 1.48309917e-18, 2.43162246e-20, 0.00000000e+00,
       4.99705236e-20, 1.44815827e-13, 1.40344154e-19, 2.59100847e-08,
       8.48125112e-15, 3.26889341e-28])

In [9]:
print(X_train[0])
print(len(X_train[0]))
len(X_train)

[[3.75514827e-20]
 [2.56857822e-13]
 [8.81158303e-06]
 ...
 [1.65553749e-04]
 [7.10123422e-05]
 [1.46182065e-04]]
55040


90

In [10]:
print(Y_train)
len(Y_train)

[3.75514827e-20 1.48309917e-18 2.43162246e-20 0.00000000e+00
 4.99705236e-20 1.44815827e-13 1.40344154e-19 2.59100847e-08
 8.48125112e-15 3.26889341e-28 1.95812776e-21 1.16014810e-13
 1.32072713e-18 6.05673973e-22 1.35988112e-17 8.37287767e-18
 2.33854678e-07 2.05902020e-14 3.31584905e-14 1.09061954e-20
 1.59242089e-27 2.05037332e-18 2.10388488e-18 3.35791846e-14
 6.75671035e-17 1.75250659e-14 2.07558964e-19 2.94831886e-17
 3.54930859e-17 1.04178338e-19 9.45902162e-20 9.17274475e-20
 1.70337284e-19 1.37817521e-14 6.39614100e-18 2.41209798e-28
 1.91091672e-19 2.10372982e-14 8.79957514e-20 8.01383240e-21
 1.21626350e-14 0.00000000e+00 1.62386762e-15 4.89418212e-14
 1.24420458e-14 4.09525422e-16 2.33794452e-18 6.78193197e-02
 1.98685003e-15 1.71899295e-20 4.18970533e-15 1.97536596e-15
 4.75129698e-17 3.69101424e-13 3.22405249e-08 2.33558036e-17
 1.15213625e-20 1.01794886e-14 1.89392513e-21 6.02219514e-17
 5.01210557e-28 7.74817288e-19 8.24923100e-15 1.10421015e-15
 5.46875809e-11 6.331074

90

In [11]:
X_valid = []
Y_valid = []
for i in valid:
    X_valid.append(i[0])
    Y_valid.append(i[0][0])
X_valid=np.array(X_valid)
Y_valid=np.array(Y_valid)

In [12]:
do_training = True
batch_size = 100
epochs = 10

saved_weights_fname = "./psMNIST-weights.hdf5"
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=saved_weights_fname, monitor="val_loss", verbose=1, save_best_only=True
    ),
]

result = None
if do_training:
    result = model.fit(
        X_train,
        Y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_valid, Y_valid),
        callbacks=callbacks,
    )

Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 4.4939 - accuracy: 0.0000e+00
Epoch 1: val_loss improved from inf to 4.48830, saving model to ./psMNIST-weights.hdf5
1/1 [==============================] - 184s 184s/step - loss: 4.4939 - accuracy: 0.0000e+00 - val_loss: 4.4883 - val_accuracy: 0.0000e+00
Epoch 2/10
